In [7]:
import numpy as np
import os
from src.utils_py.io.gro import read_gro, write_gro
from src.utils_py.geom.Box import Box
from src.utils_py.geom.Droplet import Droplet
from src.utils_py.geom.Doughnut import Doughnut
from src.utils_py.geom.Worm import Worm
from src.utils_py.geom.Roll import Roll
from src.utils_py.geom.Perforation import Perforation
from src.utils_py.assembler.build import build_system
from src.utils_py.gro.Structure import Structure

---

# Тесты капелек

In [8]:
''' ПАРАМЕТРЫ НЕ ТРОГАТЬ!!! '''
insertion_limit = int(1e5)
rotation_limit = 1000
package = 0.3
distance = {'min': 0.08**2, 'opt': 0.12**2}

In [58]:
shape_type = 'doughnut'

WIDTH_X, WIDTH_Y, H = 15, 15, 7.5
phi = 0.2
th = np.deg2rad(180)

In [59]:
box = np.array([WIDTH_X, WIDTH_Y, H])

structure = Structure(
    title='Points',
    box=box,
    atoms=np.empty(0, dtype=object),
    atoms_xyz=np.zeros((0, 3))
)

points = structure.atoms_xyz

names = ['point']
density = [12] # nm-3 Плотность, как у воды

# XZ surface
region = Doughnut(
    center = np.array([WIDTH_X / 2, WIDTH_Y / 2, H / 2]),
    borders = np.array([WIDTH_X, WIDTH_Y, H]),
    phi = phi,
    th = th,
    H = H
)

insert_shapes = [region]
shapes = [region]
numbers = list(np.round(np.array([shapes[i].get_volume() * density[i] for i in range(len(names))])).astype(int))

structure = build_system(
    './', structure, names, numbers, insert_shapes, points,
    insertion_limit=insertion_limit,
    rotation_limit=rotation_limit,
    package=package,
    min_dist2=distance['min']
)

filename = '1'

os.system(f'mkdir synthetic/{shape_type}')

with open(f'synthetic/{shape_type}/{filename}.gro', 'w') as f:
    f.write(write_gro(structure))

Number of molecules:
point 	 4050

Filling system:


100%|██████████| 4050/4050 [00:01<00:00, 3189.84it/s]
mkdir: synthetic/doughnut: File exists


---

# Генерация искуственных датасетов

In [9]:
regions = [ Droplet ,  Doughnut ,  Worm ,  Roll ,  Perforation]
types   = ['droplet', 'doughnut', 'worm', 'roll', 'perforation']
phi     = np.array([0.1, 0.2, 0.3, 0.5, 0.8])
WIDTH_X = np.array([15 , 15 , 15 , 15 , 15 ])
WIDTH_Y = np.array([15 , 15 , 15 , 15 , 15 ])
H       = np.array([7.5, 7.5, 7.5, 5  , 7.5])
c  = H / 2
c[0] = 0
c[2] = 0
thetas    = [180, 170, 160, 150, 140, 130, 120, 110, 100, 90.1]

In [10]:
iterations = 10

In [ ]:
for th in thetas:
    for id in range(len(regions)):
    # for id in [1]:
        box = np.array([WIDTH_X[id], WIDTH_Y[id], H[id]])

        names = ['point']
        density = [12] # nm-3 Плотность, как у воды

        for i in range(iterations, iterations+10):
            structure = Structure(
                title='Points',
                box=box,
                atoms=np.empty(0, dtype=object),
                atoms_xyz=np.zeros((0, 3))
            )
            points = structure.atoms_xyz

            # XZ surface
            region = regions[id](
                center = np.array([WIDTH_X[id] / 2, WIDTH_Y[id] / 2, c[id]]),
                borders = np.array([WIDTH_X[id], WIDTH_Y[id], H[id]]),
                phi = phi[id],
                th = np.deg2rad(th),
                H = H[id]
            )

            insert_shapes = [region]
            shapes = [region]
            numbers = list(np.round(np.array([shapes[i].get_volume() * density[i] for i in range(len(names))])).astype(int))

            structure = build_system(
                './', structure, names, numbers, insert_shapes, points,
                insertion_limit=insertion_limit,
                rotation_limit=rotation_limit,
                package=package,
                min_dist2=distance['min']
            )

            os.system(f'mkdir synthetic/{th}')
            os.system(f'mkdir synthetic/{th}/{types[id]}')

            with open(f'synthetic/{th}/{types[id]}/{i+1}.gro', 'w') as f:
                f.write(write_gro(structure))

---

## Тесты капелек alpha

In [1]:
import importlib
import numpy as np
from scipy.stats import norm
from tqdm import tqdm
import os

import sys
sys.path.append("..")  # Avoid error with importing of src
from src.utils_py.io.gro import read_gro, write_gro
from src.utils_py.geom.Box import Box
from src.utils_py.geom import interface_geom
from src.utils_py.assembler.build import build_system
from src.utils_py.gro.Structure import Structure

In [2]:
shapes = ["Droplet", "Doughnut", "Worm", "Roll", "Perforation", "Layer"]
shapes_iter = [40, 10, 20, 10, 10, 2]

In [3]:
box = np.array([10, 10, 10])

L = np.arange(1, 4.1, 1)
print("L", L)
PHI = np.arange(0.05, 1, 0.1)
print("PHI", PHI)
THETA = np.arange(90, 181, 10)
THETA[0] = 91
print("THETA", THETA)
DELTA = np.arange(0, 0.21, 0.05)
print("DELTA", DELTA)

L [1. 2. 3. 4.]
PHI [0.05 0.15 0.25 0.35 0.45 0.55 0.65 0.75 0.85 0.95]
THETA [ 91 100 110 120 130 140 150 160 170 180]
DELTA [0.   0.05 0.1  0.15 0.2 ]


In [4]:
def gen_range(points: np.array, step: float,  iter: int, lower: float):
    POINTS = []
    dist = norm(loc=0, scale=step / 6)
    for point in points:
        point_arr = []

        for j in range(iter):
            val = point + dist.rvs(1).item()
            while val < lower:
                val = point + dist.rvs(1).item()

            point_arr.append(val)

        POINTS.append(point_arr)

    return np.array(POINTS)

In [4]:
def add_noise(point: float, step: float, lower: float):
    dist = norm(loc=0, scale=step / 6)
    val = point + dist.rvs(1).item()
    while val < lower:
        val = point + dist.rvs(1).item()

    return val

In [ ]:
for i, shape in enumerate(shapes):
    iter = shaper_iter[i]

    L_shape = []
    dl = 1
    dist = norm(loc=0, scale=dl / 6)
    for l in L:
        l_shape = []
        for j in range(iter):
            l_shape.append(l + dist.rvs(1).item())
        L_shape.append(l_shape)

    print('L_shape')
    print(np.array(L_shape))

    PHI_shape = []
    dphi = 0.1
    dist = norm(loc=0, scale=dphi / 6)
    for phi in PHI:
        phi_shape = []
        for j in range(iter):
            val = phi + dist.rvs(1).item()
            while val < 0:
                val = phi + dist.rvs(1).item()

            phi_shape.append(val)
        PHI_shape.append(phi_shape)

    print('PHI_shape')
    print(np.array(PHI_shape))

    THETA_shape = []
    dtheta = 10
    dist = norm(loc=0, scale=dtheta / 6)
    for theta in THETA:
        theta_shape = []
        for j in range(iter):
            val = theta + dist.rvs(1).item()
            while val < 91:
                val = theta + dist.rvs(1).item()

            theta_shape.append(val)
        THETA_shape.append(theta_shape)

    print('THETA_shape')
    print(np.array(THETA_shape))

    DELTA_shape = []
    ddelta = 0.05
    dist = norm(loc=0, scale=ddelta / 6)
    for delta in DELTA:
        delta_shape = []
        for j in range(iter):
            val = delta + dist.rvs(1).item()
            while val < 0:
                val = delta + dist.rvs(1).item()

            delta_shape.append(val)
        DELTA_shape.append(delta_shape)

    print('DELTA_shape')
    print(np.array(DELTA_shape))

In [27]:
counter = 0
shapes_counter = dict(zip(shapes, [0 for i in range(len(shapes))]))

for i, shape in enumerate(shapes):
    L_range = gen_range(L, 1, shapes_iter[i], 1)
    PHI_range = gen_range(PHI, 0.1, shapes_iter[i], 0.05)
    THETA_range = gen_range(THETA, 10, shapes_iter[i], 91)
    DELTA_range = gen_range(DELTA, 0.05, shapes_iter[i], 0)

    module = importlib.import_module(f"src.utils_py.geom.interface_geom.{shape.title()}")
    Interface = getattr(module, shape.title())

    for j in tqdm(range(shapes_iter[i]), desc="iter"):
        for l in L_range[:, j]:
            for phi in PHI_range[:, j]:
                for theta in THETA_range[:, j]:
                    for delta in DELTA_range[:, j]:
                        box[2] = box[0] / l

                        region = Interface(
                            box / 2,
                            box,
                            l,
                            phi,
                            theta=np.deg2rad(theta),
                            delta=delta,
                            extention="alpha",
                        )

                        if region.check_existence():
                            counter += 1
                            shapes_counter[shape] += 1

print(f"Total number of conformations {counter}")

iter: 100%|██████████| 2/2 [00:00<00:00, 45.58it/s]

Total number of conformations 59459


In [5]:
counter = 0
shapes_counter = dict(zip(shapes, [0 for i in range(len(shapes))]))

for i, shape in enumerate(shapes):
    module = importlib.import_module(f"src.utils_py.geom.interface_geom.{shape.title()}")
    Interface = getattr(module, shape.title())

    for j in tqdm(range(shapes_iter[i]), desc="iter"):
        for l in L:
            for phi in PHI:
                for theta in THETA:
                    for delta in DELTA:
                        l_noise = add_noise(l, 1, 1)
                        phi_noise = add_noise(phi, 0.1, 0.05)
                        theta_noise = add_noise(theta, 10, 91)
                        delta_noise = add_noise(delta, 0.05, 0)

                        box[2] = box[0] / l

                        region = Interface(
                            box / 2,
                            box,
                            l=l_noise,
                            phi=phi_noise,
                            theta=np.deg2rad(theta_noise),
                            delta=delta_noise,
                            extention="alpha",
                        )

                        if region.check_existence():
                            counter += 1
                            shapes_counter[shape] += 1

print(f"Total number of conformations {counter}")

iter:  10%|█         | 1/10 [00:07<01:04,  7.20s/it]


KeyboardInterrupt: 

In [28]:
shapes_counter

{'Droplet': 9709,
 'Doughnut': 10981,
 'Worm': 12523,
 'Roll': 13391,
 'Perforation': 9655,
 'Layer': 3200}

In [26]:
sum(shapes_counter.values())

59596

In [16]:
270 * 4 + 1085 + 646 * 2 + 1315 + 955 + 1600 // 4

6127

In [19]:
270 * 4 * 10 + 1085 * 10 + 646 * 2 * 10 + 1315 * 10 + 955 * 10 + 1600 * 10 // 4

61270

In [8]:
1600 * 8

12800

In [2]:
646 * 2 * 8

10336